In [ ]:
import pickle
import cv2
import os
import numpy as np
import imutils
import time


BASE_DIR = os.path.dirname('cv')
print("[INFO] BASE DIR: ", BASE_DIR)

In [ ]:
# load our serialized face detector from disk
print("[INFO] loading face detector...")
protoPath = os.path.join(BASE_DIR, "face_detection_model/deploy.prototxt")
modelPath = os.path.join(BASE_DIR, "face_detection_model/res10_300x300_ssd_iter_140000.caffemodel")
detector = cv2.dnn.readNetFromCaffe(protoPath, modelPath)

In [ ]:
# load our serialized face embedding model from disk
embedding_model = os.path.join(BASE_DIR, 'openface_nn4.small2.v1.t7')
print("[INFO] loading face recognizer...")
embedder = cv2.dnn.readNetFromTorch(embedding_model)

In [ ]:
# load the actual face recognition model along with the label encoder
recognizer_file = os.path.join(BASE_DIR, 'output/recognizer.pickle')
le_file = os.path.join(BASE_DIR, 'output/le.pickle')
recognizer = pickle.loads(open(recognizer_file, "rb").read())
le = pickle.loads(open(le_file, "rb").read())

In [6]:
print("[INFO] starting video stream...")
cap = cv2.VideoCapture(0)


while (True):
    ret, frame = cap.read()

    frame = imutils.resize(frame, width=600)
    (h, w) = frame.shape[:2]

    # construct a blob from the image
    imageBlob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 1.0, (300, 300),
        (104.0, 177.0, 123.0), swapRB=False, crop=False)

    # apply OpenCV's deep learning-based face detector to localize
    # faces in the input image
    detector.setInput(imageBlob)
    detections = detector.forward()

    # loop over the detections
    for i in range(0, detections.shape[2]):
        # extract the confidence (i.e., probability) associated with
        # the prediction
        confidence = detections[0, 0, i, 2]
        #print("[DEBUG] confidence: ", float(confidence))

        # filter out weak detections
        if confidence > 0.45:
            # compute the (x, y)-coordinates of the bounding box for
            # the face
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")

            # extract the face ROI
            face = frame[startY:endY, startX:endX]
            (fH, fW) = face.shape[:2]

            # ensure the face width and height are sufficiently large
            if fW < 20 or fH < 20:
                continue

            # construct a blob for the face ROI, then pass the blob
            # through our face embedding model to obtain the 128-d
            # quantification of the face
            faceBlob = cv2.dnn.blobFromImage(face, 1.0 / 255, (96, 96), (0, 0, 0), swapRB=True, crop=False)
            embedder.setInput(faceBlob)
            vec = embedder.forward()

            # perform classification to recognize the face
            preds = recognizer.predict_proba(vec)[0]
            j = np.argmax(preds)
            proba = preds[j]
            name = le.classes_[j]

            # draw the bounding box of the face along with the
            # associated probability
            text = "{}: {:.2f}%".format(name, proba * 100)
            y = startY - 10 if startY - 10 > 10 else startY + 10
            
            cv2.rectangle(frame, (startX, startY), (endX, endY), (0, 0, 255), 2)
            cv2.putText(frame, text, (startX, y), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 0, 255), 2)


    cv2.imshow('Camera', frame)
    k = cv2.waitKey(100) & 0xff # Press 'ESC' for exiting video
    if k == 27:
        break

cap.release()
cv2.destroyAllWIndows()